# PDFの解析を行うチャットボットの作成


本レポートでは、chainlitとlangchainを利用して、PDFを分析するチャットボットの作成についてまとめる。

## chainlitとは？
本番環境に対応した対話型AIのUIを作るためのライブラリ

## langchainとは？
大規模言語モデルを利用してアプリケーションを作成するためのフレームワーク。外部のデータベースや言語処理系と組み合わせて、高度な処理を行うアプリケーションを作成することができる。

いずれもpip installによってインストールを行う。


## アプリケーションの動かし方

- 本ドキュメントの最下部に記述されたコードを、任意の.pyファイルに貼り付ける。

- 次に、.env というファイルを作成し、その中に次のようにOpenAIのapiキーを記述する。

	OPENAI_API_KEY=あなたのOpenAI APIキー








- chainlitを起動するためには、.pyファイルが位置するディレクトリにおいて、以下のコマンドを実行する。

	chainlit run ファイル名 -w

上記の手順が完了すれば、アプリケーションがブラウザ上で起動するはずである。ここで任意のPDFファイルを渡し、質問を行うことができる。

## コードの解説



In [2]:
# ライブラリインポート
import dotenv
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import chainlit as cl

### APIキーの取得

まず、dotenvを利用し、.envファイルからapiキーを変数名とともに辞書に格納する。

この変数に対し、.get("OPENAI_API_KEY", "")でapiキーのみを取得できる

In [3]:

env_values = dotenv.dotenv_values()
print(env_values)
print("\napiキーを抽出:\n", env_values.get("OPENAI_API_KEY", ""))

OrderedDict({'OPENAI_API_KEY': 'hoge'})

apiキーを抽出:
 hoge


### プロンプトテンプレート

プロンプトテンプレートとは、言語モデルに入力するプロンプト生成のために事前に定義されるもの。( from langchain.prompts import PromptTemplate )


In [4]:
# プロンプトを定義
prompt = PromptTemplate(
    template="""
    文章を前提にして質問に答えてください。

    文章 :
    {document}

    質問 : {question}
    """,
    input_variables=["document", "question"],
)


## ソースコード

In [ ]:
import dotenv
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import chainlit as cl

env_values = dotenv.dotenv_values()

# プロンプトを定義
prompt = PromptTemplate(
    template="""
    文章を前提にして質問に答えてください。

    文章 :
    {document}

    質問 : {question}
    """,
    input_variables=["document", "question"],
)



@cl.on_chat_start
async def on_chat_start():
	"""初回起動時に呼び出される."""

	# PDFを読み込む処理
	files = None

	# awaitメソッドのために、whileを利用する。アップロードされるまで続く。
	while files is None:
		# chainlitの機能に、ファイルをアップロードさせるメソッドがある。
		files = await cl.AskFileMessage(
			# ファイルの最大サイズ
			max_size_mb=20,
			# ファイルをアップロードさせる画面のメッセージ
			content="PDFを選択してください。",
			# PDFファイルを指定する
			accept=["application/pdf"],
			# タイムアウトなし
			raise_on_timeout=False,
		).send()

	file = files[0]

	# アップロードされたファイルのパスから中身を読み込む。
	documents = PyMuPDFLoader(file.path).load()
    
    # PDFを分割する処理
	text_splitter = SpacyTextSplitter(chunk_size=400, pipeline="ja_core_news_sm")
	splitted_documents = text_splitter.split_documents(documents)

    # PDFの内容をベクトル化して保存する処理
	# テキストをベクトル化するOpenAIのモデル
	embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=env_values.get("OPENAI_API_KEY", ""))

	# Chromaにembedding APIを指定して、初期化する。
	database = Chroma(embedding_function=embeddings)

	# PDFから内容を分割されたドキュメントを保存する。
	database.add_documents(splitted_documents)

	# 今回は、簡易化のためセッションに保存する。
	cl.user_session.set("data", database)




@cl.on_message
async def on_message(input_message: cl.Message):
	"""メッセージが送られるたびに呼び出される."""
    
	# チャット用のOpenAIのモデル
	open_ai = ChatOpenAI(api_key=env_values.get("OPENAI_API_KEY", ""), model="gpt-4") # type: ignore

	# セッションからベクトルストアを取得（この中にPDFの内容がベクトル化されたものが格納されている）
	database = cl.user_session.get("data")

	# 質問された文から似た文字列を、DBより抽出
	documents = database.similarity_search(input_message.content)

	# 抽出したものを結合
	documents_string = ""
	for document in documents:
		documents_string += f"""
		---------------------------------------------
		{document.page_content}
		"""

	# プロンプトに埋め込みながらOpenAIに送信
	result = open_ai(
		[
			HumanMessage(
				content=prompt.format(
					document = database,
					# query = input_message.content,
					question = f"{input_message.content}: {documents}", # メッセージと一緒にドキュメントの内容も送信
				)
			)
		]
	).content

	await cl.Message(content=result, author="Answer").send()


### 参考サイト

- [chainlit公式サイト](https://docs.chainlit.io/get-started/overview)

- [ローカルで気軽にRAGを使って会話することが簡単すぎてビビった。](https://qiita.com/mitsumizo/items/469d79c5e81d9189a9e4)